In [1]:
%load_ext autoreload
%autoreload 2

import pickle

import numpy as np
from tqdm.notebook import tqdm
import matplotlib as plt

import sys
sys.path.append('..')

from src.datasets import load_dataset
from src.evaluation.evaluation import downvote_seen_items, topn_recommendations, model_evaluate
from src.models import NGCF, ALS, eALS, iALS

In [2]:
data_path = '../data'

training_yelp, testset_yelp, holdout_yelp, training_matrix_yelp, data_description_yelp = load_dataset('yelp', data_path)
training_movielens, testset_movielens, holdout_movielens, training_matrix_movielens, data_description_movielens = load_dataset('movielens', data_path)

In [3]:
def bench_model(model, holdout, testset, data_description, topn=10):
    scores = model.score_users(holdout.userid.values)
    downvote_seen_items(scores, testset, data_description)
    recs = topn_recommendations(scores, topn=topn)

    hr, mrr, ndcg, cov = model_evaluate(recs, holdout, data_description, topn=topn)

    print(f'HR@{topn} = {hr:.4f}')
    print(f'MRR@{topn} = {mrr:.4f}')
    print(f'nDCG@{topn} = {ndcg:.4f}')
    print(f'COV@{topn} = {cov:.4f}')

In [4]:
# model_movielens = iALS(factors=100, device='cuda', save_path='../weights/ials_movielens.pkl').fit(training_matrix_movielens)
with open('../weights/ials_movielens.pkl', 'rb') as handle:
    model_movielens = pickle.load(handle)

In [7]:
# model_yelp = iALS(factors=100, device='cuda', save_path='../weights/ials_yelp.pkl').fit(training_matrix_yelp)
with open('../weights/ials_yelp.pkl', 'rb') as handle:
    model_yelp = pickle.load(handle)

In [8]:
print('MovieLens:')
bench_model(model_movielens, holdout_movielens, testset_movielens, data_description_movielens)

MovieLens:
HR@10 = 0.1013
MRR@10 = 0.0354
nDCG@10 = 0.0506
COV@10 = 0.3661


In [9]:
print('Yelp:')
bench_model(model_yelp, holdout_yelp, testset_yelp, data_description_yelp)

Yelp:
HR@10 = 0.0409
MRR@10 = 0.0152
nDCG@10 = 0.0211
COV@10 = 0.0619


In [10]:
# import json
# import torch

# with open('../configs/ngcf_base.json') as f:
#     config = json.load(f)

# n_users, n_items = training_matrix_movielens.shape
# config['model_args']['n_users'] = n_users
# config['model_args']['n_items'] = n_items

# model = NGCF(**config['model_args'])
# model.load_state_dict(torch.load('../lightning_logs/version_30/checkpoints/epoch=399-step=2399.ckpt')['state_dict'])
# model.eval();

In [9]:
import implicit


topn = 10
als = implicit.als.AlternatingLeastSquares(iterations=100, regularization=0.1, num_threads=-1)
als.fit(training_matrix_movielens)

recs_movielens = als.recommend_all(training_matrix_movielens, filter_already_liked_items=True, N=topn)
hr, mrr, ndcg, cov = model_evaluate(recs_movielens, holdout_movielens, data_description_movielens, topn=topn)

print(f'HR@{topn} = {hr:.4f}')
print(f'MRR@{topn} = {mrr:.4f}')
print(f'nDCG@{topn} = {ndcg:.4f}')
print(f'COV@{topn} = {cov:.4f}')

  0%|          | 0/100 [00:00<?, ?it/s]

HR@10 = 0.0947
MRR@10 = 0.0337
nDCG@10 = 0.0478
COV@10 = 0.4182
